# 準備

## ライブラリインポート

In [ ]:
import json
import base64
import os
from PIL import Image
import boto3
import botocore

boto3_bedrock = boto3.client(service_name='bedrock-runtime',region_name='us-east-1')


## util 関数

### 入力画像のバリデーション
[Anthropic Claude の inference parameter](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html#model-parameters-anthropic-claude-messages-request-response) では以下の制約がある

> data – (required) The base64 encoded image bytes for the image. The maximum image size is 3.75MB. The maximum height and width of an image is 8000 pixels. 

この制約を満たさないと以下のエラーが出る

> ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: messages.0.content.0.image.source.base64.data: Image format image/jpeg not supported

したがって base64 encode した画像サイズと最大の height/width が 8000 ピクセルより小さいかバリデーションする必要がある

In [ ]:
def encode_image_for_claude(image_path):
    """
    Claude3 への入力用に画像を base64 エンコードする

    Anthropic Claude の inference parameter には画像の高さ、幅、サイズの制限があります。
    制限を満たさない場合は ValueError を返します。制限を満たせば base64 でエンコードされた画像データを返します。

    Args:
        image_path (str): 入力画像への相対パス

    Returns:
        str: base64 でエンコードされた画像データ

    Raises:
        ValueError: 入力画像の高さ、幅、サイズの制限を超過した場合

    Examples:
        >>> encode_image_for_claude("path_to_image")
        "base64_encoded_image"
    """
    file_size_bytes = os.path.getsize(image_path)
    file_size_mb = file_size_bytes / 1000 / 1000
    if file_size_mb > 3.75:
        raise ValueError("Image size exceeded 3.75 MB quota")

    width, height = Image.open(image_path).size
    if width > 8000:
        raise ValueError("Image width exceeded 8000 pixel quota")
    if height > 8000:
        raise ValueError("Image height exceeded 8000 pixel quota")

    with open(image_path, "rb") as image_file:
        image_bytes = image_file.read()
    encoded_image = base64.b64encode(image_bytes).decode("utf-8")
    return encoded_image

In [ ]:
def retrieve_content_type_from_image(image_path):
    """
    画像の種類(png か jpeg か)を判別する

    Anthropic Claude の inference parameter には media_type が引数にある
    この media_type を画像のバイト列から判定する
    判定方法はhttps://xaro.hatenablog.jp/entry/2017/05/17/103000 を参照

    Args:
        image_path (str): 入力画像への相対パス

    Returns:
        str: media_type. "image/png" または "image/jpeg"

    Raises:
        ValueError: png でも jpeg でもない場合

    Examples:
        >>> retrieve_content_type_from_image("path_to_image.png")
        "image/png"
    """
    with open(image_path, "rb") as image_file:
        image_bytes = image_file.read()

    if image_bytes.startswith(b"\x89PNG\r\n\x1a\n"):
        return "image/png"
    elif image_bytes.startswith(b"\xff\xd8"):
        return "image/jpeg"
    else:
        raise ValueError("Image is not png nor jpeg")

### 一つの画像を入力とするプロンプト

In [ ]:
def invoke_claude_with_image(prompt_image_path,prompt_text):
    # Claude3 Haiku を使用する
    modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
    contentType = 'application/json'
    accept = 'application/json'
    outputText = "\n"
    try:
        media_type = retrieve_content_type_from_image(prompt_image_path)
    except ValueError as e:
        print(e)
        return None    
    try:
        encoded_image = encode_image_for_claude(prompt_image_path)
    except ValueError as e:
        print(e)
        return None
    body = json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 2000,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": media_type,
                                "data": encoded_image,
                            },
                        },
                        {"type": "text", "text": prompt_text},
                    ],
                }
            ],
        }
    )
    try:
        response = boto3_bedrock.invoke_model(body=body, modelId=modelId,contentType=contentType,accept=accept)
        response_body = json.loads(response.get('body').read())
        output_text = response_body.get('content')[0].get('text')
        return output_text

    except botocore.exceptions.ClientError as error:

        if error.response['Error']['Code'] == 'AccessDeniedException':
               print(f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubeshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

        else:
            raise error